# ADM Homework 3

This homework requires a number of long-running tasks to be executed one after the other.

The work is mostly presented through a series of Python scripts, each one executing one processing step and implementing its own error recovery strategy.

This notebook will link the scripts in order, to make it easier to follow what's happening.

## Downloading top charts ([script](download_top_charts.py))

The first step in the homework is to download all the pages in MyAnimeList's top anime charts. The HW requires the first 20K animes to be downloaded, but MAL as a whole only contains ~19K.

The top charts' pages are saved in a directory one after the other. In case of errors, it is possible to re-launch the script and the download will resume seamlessly, as the number of successfully downloaded files can be used to compute the page on which an error happened.

This same strategy will be used in all the scripts in which an error may happen in expensive tasks.

## Extracting anime URLs ([script](extract_anime_urls.py))

Now that we downloaded all the top charts' pages, we simply parse them to extract all the URLs, which are then saved to a TXT file.

## Downloading all individual anime's pages ([script](download_anime.py))

We iterate the URLs that we retrieved in the previous step, and download them all. Since we're swarming the server with requests, the site will start rate-limiting us.

I just implemented a retry-strategy that waits some time after each error and then simply tries again, including an incremental backoff to avoid an excessive load when failures start happening.

## Parsing pages ([script](parse_pages.py))

Here we extract all the required information from the pages. I took a look at the page's source code to try and find the simplest BeautifulSoup query to extract the information and wrapped it with safety checks to deal with possibly unexpected formats.

For each anime, a TSV has been generated according to the specification.

## Extracting search info ([script](extract_search_info.py))

This is a post-processing step on the previous script, to retain only info needed in the search engine (title, NLTK-processed synopsis and URL).

Synopsis processing involves removing stopwords and punctuation and running lemmatization jointly with part-of-speech tagging. Here this approach was chosen to try and get more meaningful results, trying to leverage the text's semantics, compared to simple stemming.

For simplicity, for each anime a file is produced where the first line denotes the title, the second the URL, each of the following ones is a lemma from the synopsis.

## Create the vocabulary ([script](create_vocabulary.py))

Create the vocabulary. It is a txt file where each line denotes a word, the ID being implied by the word's ordering (first word is 0, second is 1 and so on).

## Create the first inverted index ([script](create_first_index.py)

Now that we have extracted the synopses from each anime, lemmatized them and build a vocabulary, the next step is to create the index mapping each term to the documents it appears in.

## Start running queries!

Now we can show some results here. We built the index, let's use it to run a query.

In [1]:
from first_index_utils import run_query_on_first_index

print(run_query_on_first_index(["saiyan"]))

[('Dragon Ball Z Movie 10: Kiken na Futari! Super Senshi wa Nemurenai', 'After his loss to Goku, Broly crash lands and hibernates on earth. After some time, he is awakened by Trunks and Goten, who Broly believes is Kakarott, and goes on a rampage to kill both of them. At the same time, Gohan is on his way to challenge the Legendary Super Saiyan alone.', 'https://myanimelist.net/anime/903/Dragon_Ball_Z_Movie_10__Kiken_na_Futari_Super_Senshi_wa_Nemurenai'), ('Dragon Ball Super', 'Seven years after the events of\n              \n              , Earth is at peace, and its people live free from any dangers lurking in the universe. However, this peace is short-lived; a sleeping evil awakens in the dark reaches of the galaxy: Beerus, the ruthless God of Destruction.\n              \n\n              Disturbed by a prophecy that he will be defeated by a "Super Saiyan God," Beerus and his angelic attendant Whis start searching the universe for this mysterious being. Before long, they reach Earth

"Saiyan" yields plenty of Dragon Ball-related results, yay!

# Algorithmic Question

The problem is a classic example in which dynamic programming can be used to find a solution.

There are some instances whose solution is trivial, namely:
- If the array has no element, then the maximum possible duration is 0;
- If the array has just one element, then the mamixum possible duration is the value of its only element.

Let's see what happens if the array is longer:
- With an array of length 2, you pick the biggest of the two elements;
- With an array of length 3, you either pick the second element, or the sum of the first and third, whichever is higher;
- With an array of length 4, there are much more possible choices.

Luckily, the first two cases can be used to design a recursive solution.

The trainer goes over his appointments in order. For each one, say the n-th, he can either:
- Accept the appointment, in which case the next possible appointment he can accept is the (n+2)-th;
- Decline the appointment, in which case the next possible appointment he can accept is the (n+1)-th.

The algorithm goes over each appointment, and at each step computes the highest value it can reach both by accepting or declining the appointment.
Depending on the choice, it recurses checking the max value for the sub-array starting at position either n+1 or n+2.

In [1]:
def solution(arr):
    # Base cases
    if len(arr) == 0:
        return []
    elif len(arr) == 1:
        return [arr[0]]

    max_if_accept = [arr[0]] + solution(arr[2:])
    max_if_decline = solution(arr[1:])

    return (
        max_if_accept
        if sum(max_if_accept) >= sum(max_if_decline)
        else max_if_decline
    )

Let's run the algorithm on the example from the homework:

In [4]:
solution([30, 40, 25, 50, 30, 20])

[40, 50, 20]

Yay, it works!

Now here's a snippet to try it yourself:

In [6]:
print('Insert the lengths of the appointments (e.g. "1, 2, 3")')
array = list(map(int, input().split(', ')))

print(f"Your input: {array}")
print(f"Solution: {solution(array)}")


Insert the lengths of the appointments (e.g. "1, 2, 3")
Your input: [30, 40, 25, 50, 30, 20]
Solution: [40, 50, 20]
